In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/NYU_Spring_2021/CCM/code/eccentricity

Mounted at /content/drive
/content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import re
import sys
import tarfile

from six.moves import urllib
import numpy as np


In [10]:
contrast_norm = None

def read_and_decode(filename_queue):
  reader = tf.TFRecordReader()
  _, serialized_example = reader.read(filename_queue)
  features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
          'image_raw': tf.FixedLenFeature([], tf.string),
          'label': tf.FixedLenFeature([], tf.int64),
      })

  if contrast_norm == 'areafactor':
      image = tf.decode_raw(features['image_raw'], tf.float32)
  else:
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image = tf.cast(image, tf.float32) * (1. / 255)
 
  image.set_shape(np.prod([5, 60, 60]))
  image = tf.reshape(image, [5, 60, 60, 1])
  image = image - 0.5

  # Convert label from a scalar uint8 tensor to an int32 scalar.
  label = tf.cast(features['label'], tf.int32)

  return image, label

In [1]:
def get_inputs(name, batch_size, num_epochs):
  """Reads input data num_epochs times.
  Args:
    train: Selects between the training (True) and test (False) data.
    batch_size: Number of examples per returned batch.
    num_epochs: Number of times to read the input data, or 0/None to
       train forever.
  Returns:
    A tuple (images, labels), where:
    * images is a float tensor with shape [batch_size, num_scales, crop_size, crop_size]
    * labels is an int32 tensor with shape [batch_size] with the true label,
      a number in the range [0, NUM_CLASSES()).
    Note that an tf.train.QueueRunner is added to the graph, which
    must be run using e.g. tf.train.start_queue_runners().
  """
  if not num_epochs: num_epochs = None
  filename = 'experiments/demo-experiment/datasets/gen_fl_0/train_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'

  with tf.name_scope('input'):
    filename_queue = tf.train.string_input_producer(
        [filename], num_epochs=num_epochs)

    # Even when reading in multiple threads, share the filename
    # queue.
    image, label = read_and_decode(filename_queue)

    # Shuffle the examples and collect them into batch_size batches.
    # (Internally uses a RandomShuffleQueue.)
    # We run this in two threads to avoid being a bottleneck.
    images, sparse_labels = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=8,
        capacity=1000 + 3 * batch_size,
        # Ensures a minimum amount of shuffling of examples.
        min_after_dequeue=1000)

    return images, sparse_labels

In [12]:
images, labels = get_inputs('train', 128, 20)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.shuffle(min_after_dequeue).batch(batch_size)`.


In [3]:
!python src/python/ecc/ecc_train.py --parity=even --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --train_dir=experiments/demo-experiment/gen_0  --contrast_norm=None  --model_name=5-1-1-1-1_total_pool_contrast_None_lr0.01



W0511 21:17:28.371506 139648611170176 module_wrapper.py:139] From src/python/ecc/ecc_train.py:130: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0511 21:17:30.157699 139648611170176 module_wrapper.py:139] From src/python/ecc/ecc_train.py:131: The name tf.gfile.DeleteRecursively is deprecated. Please use tf.io.gfile.rmtree instead.


W0511 21:17:30.388922 139648611170176 module_wrapper.py:139] From src/python/ecc/ecc_train.py:132: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0511 21:17:30.406663 139648611170176 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from t

In [7]:
!python src/python/ecc/ecc_eval.py --parity=even --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0511 21:51:00.538238 140528967473024 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [6]:
 

!python src/python/ecc/ecc_eval.py --parity=even --flanker_type=1 --flanker_ecc=240 --target_ecc=120 --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None  



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 03:48:46.139373 140591339014016 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [12]:
!python src/python/ecc/ecc_eval.py  --parity=even --flanker_type=1 --flanker_ecc=480 --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None  



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 04:10:18.679971 139639418263424 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [14]:
!python src/python/ecc/ecc_eval.py  --parity=even --flanker_type=2 --flanker_ecc=480 --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None  



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 04:18:27.242351 139658993899392 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [15]:
!python src/python/ecc/ecc_eval.py --parity=even --flanker_type=1 --flanker_ecc=240 --target_ecc=120 --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None  



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 04:26:32.709966 139666939029376 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [4]:
!python src/python/ecc/ecc_eval.py --parity=even --flanker_type=1 --flanker_ecc=360 --target_ecc=120 --learning_rate=0.01 --image_save_interval=1000 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None 



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 18:36:08.901512 139814646716288 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [6]:
!python src/python/ecc/ecc_train.py --parity=even --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --train_dir=experiments/demo-experiment/gen_0  --contrast_norm=None  --model_name=5-1-1-1-1_total_pool_contrast_None_lr0.01


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 18:40:50.554900 139685612459904 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(

In [7]:
!python src/python/ecc/ecc_eval.py --parity=even --flanker_type=1 --flanker_ecc=360 --target_ecc=120 --learning_rate=0.01 --image_save_interval=1000 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None 



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 18:51:17.496979 140431016585088 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [8]:
!python src/python/ecc/ecc_eval.py --parity=even --flanker_type=1 --flanker_ecc=240 --target_ecc=120 --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None  



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 18:54:11.125574 140121042745216 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [9]:
!python src/python/ecc/ecc_eval.py  --parity=even --flanker_type=2 --flanker_ecc=480 --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None  



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 18:56:29.635150 140495972079488 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [10]:
!python src/python/ecc/ecc_eval.py  --parity=even --flanker_type=1 --flanker_ecc=480 --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True --contrast_norm=None  



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 18:59:33.710168 139955023193984 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat

In [11]:
!python src/python/ecc/ecc_eval.py --parity=even --learning_rate=0.01 --image_save_interval=10 --pm=5-1-1-1-1 --num_scales=5 --total_pool=True


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0512 19:01:20.794759 139857363298176 deprecation.py:323] From /content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity/src/python/ecc/ecc.py:106: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat